# Tools  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
Tools allow agents to 'Act' in the real world. 
Careful descriptions can help your agent discover how to use your tools.

LangChain supports many tool formats and tool sets. Here we will cover some common cases, but check the [docs](https://docs.langchain.com/oss/python/langchain/tools) for more information.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

# Declaring default model
default_model = "google_genai:gemini-2.5-flash"

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=****3d99
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


## Calculator example

In this example, the docstring and inferred arguments and argument types are used by the LLM to detetermine when and how to call the tool.

In [2]:
from typing import Literal

from langchain.tools import tool


@tool
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers."""
    print("🧮 Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [3]:
from langchain.agents import create_agent

agent = create_agent(
    model=default_model,
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

This invokes your calculator tool.

In [4]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 3.1125 * 4.1234"}]}
)
print(result["messages"][-1].content)

🧮 Invoking calculator tool
3.1125 * 4.1234 = 12.8340825


We can check the **metadata** in [LangSmith Observability](https://smith.langchain.com/public/b77bde6c-f0ad-4256-bfab-7d514ece3405/r) to see this.

The tool description can have a big impact. 
This may not  invoke your calculator tool because the inputs are integers.  (results vary from run to run)

In [5]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

🧮 Invoking calculator tool
3 multiplied by 4 is 12.


This often does not invoke the tool though the input are real numbers. (results vary from run to run)

In [6]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

🧮 Invoking calculator tool
12.0


## Adding a more detailed description
While a basic description is often sufficient, LangChain has support for enhanced descriptions. The example below uses one method: Google Style argument descriptions. Used with `parse_docstring=True`, this will parse and pass the arg descriptions to the model. You can rename the tool and change its description. This can be effective when you are sharing a standard tool but would like agent-specific instructions.

In [6]:
from typing import Literal

from langchain.tools import tool


@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "Perform basic arithmetic operations on two real numbers."
        "Use this whenever you have operations on any numbers, even if they are integers."
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers.

    Args:
        a (float): The first number.
        b (float): The second number.
        operation (Literal["add", "subtract", "multiply", "divide"]):
            The arithmetic operation to perform.

            - `"add"`: Returns the sum of `a` and `b`.
            - `"subtract"`: Returns the result of `a - b`.
            - `"multiply"`: Returns the product of `a` and `b`.
            - `"divide"`: Returns the result of `a / b`. Raises an error if `b` is zero.

    Returns:
        float: The numerical result of the specified operation.

    Raises:
        ValueError: If an invalid operation is provided or division by zero is attempted.
    """
    print("🧮  Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [7]:
from langchain.agents import create_agent

agent = create_agent(
    model=default_model,
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

In [8]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

🧮  Invoking calculator tool
3.0 * 4.0 = 12


Let's check our [LangSmith Observability trace](https://smith.langchain.com/public/7d65902c-bd3c-4fc6-bbd3-7c1d66566fda/r) to see the tool description.

In [9]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

🧮  Invoking calculator tool
3 * 4 = 12


## Try your own.
Create a tool of your own and try it here!

In [ ]:
import requests

@tool(
    "currency_quotation",
    parse_docstring=True,
    description=(
        "Consulte o câmbio de uma moeda estrangeira em relação ao Real (BRL) em uma data específica. "
        "Use isso para buscar dados históricos ou atuais de cotação."
    ),
)
def get_currency_quotation(
    currency: Literal["USD", "EUR", "AUD", "CAD", "CHF", "DKK", "GBP", "JPY", "SEK"],
    date: str
) -> dict | str:
    """
    Busca a cotação de uma moeda específica em uma data informada.

    Args:
        currency (Literal["USD", "EUR", "AUD", "CAD", "CHF", "DKK", "GBP", "JPY", "SEK"]): 
            A moeda alvo desejada.
            
            - USD: Dólar Americano
            - EUR: Euro
            - AUD: Dólar Australiano
            - CAD: Dólar Canadense
            - CHF: Franco Suíço
            - DKK: Dólar Danoês
            - GBP: Libra Esterlina
            - JPY: Yen Japonês
            - SEK: Krona Sueca
        date (str): 
            A data desejada no formato YYYY-MM-DD. 
            Dados disponíveis a partir de 28/11/1984.

    Returns:
        dict: Um dicionário contendo o valor de compra, venda e data da cotação.
        str: Mensagem de erro caso a requisição falhe.
    
    Raises:
        HTTPError: Se a requisição falhar.
    """
    print(f"💰 Consultando BrasilAPI: {currency} em {date}")
    
    url = f"https://brasilapi.com.br/api/cambio/v1/cotacao/{currency}/{date}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError:
        return f"Erro: Cotação não encontrada para {currency} em {date} (Verifique se é dia útil)."
    except Exception as e:
        return f"Erro na requisição: {str(e)}"

In [14]:
from langchain.agents import create_agent

agent = create_agent(
    model=default_model,
    tools=[get_currency_quotation],
    system_prompt="You are a helpful assistant",
)

In [18]:
result = agent.invoke({"messages": [{"role": "user", "content": "Quanto tava o Dolar no dia 25 de nov de 2025 ?"}]})
print(result["messages"][-1].content)

💰 Consultando BrasilAPI: USD em 2025-11-25
[{'type': 'text', 'text': 'No dia 25 de novembro de 2025, a cotação de fechamento PTAX do dólar foi de R$ 5,3835 para compra e R$ 5,3841 para venda. A cotação de abertura foi de R$ 5,3694 para compra e R$ 5,37 para venda.', 'extras': {'signature': 'CqAMAXLI2nx5RAV5ebbfOiyAYqEzKFZMwkwvrctFDuA9s5IMiQfXM9HzkFraNU4oUec3cNA/90pYzQO3+R0vsY32YgpYXHtSTO+FqBlZT7RdT9BBjJieBlQ/qC/vVCWb6ydwkh5ceWnUMkgcQgr9sme1ciyCsvmKdELguCCHS/FqMhxnJyWggPWJjvhsWadD3r8jqTVKMQZePl1wpXXRF9nLIahn+6vuhEsqQC74LDGzMDljz1Afj3rinn549uwosf24BH6VZV3nadvv5N9dKSxrUt715+QL0CkD6qFdt4lw3fk522B7GRZsj8zDjIH1ywYoqzmZJypB+M/lT+avXGyK145paDlXxwPEC85F1k59AyrUVMeJNxFPNsRbZdlmXJzCzxN2b+JD1rE5z2/Fy4pOTbQv7EjuREtU23ehc6b9biiOllo5XsBC3URyU7Uukpb/Ya/rS82PY+i9XVqyFMfU39cnDnvdpBYiqv9k9sVtuUAWW5h6joIhrLcIifgyT9klgdlM+NxN/+80R4RVe6VXeT3TB8ORDBxR+Wt0ZUbkgboDPdqolUa7sW9+PS1fZvu4VK7mPHk/PUWQgr3YuGTIM9yxPdlKNTpuyBJwR1Qr2BQ9a7q4lLvL6y03wp8T1ulZv0GNvOWtR+rWWAxKDn49GuUsc+jrUKeowXkiaehuSZVz1trZ3l0PzCbVh8cwrQzE

In [19]:
result = agent.invoke({"messages": [{"role": "user", "content": "Quanto tava o euro no dia 25 de nov de 2025 ?"}]})
print(result["messages"][-1].content)

💰 Consultando BrasilAPI: EUR em 2025-11-25
[{'type': 'text', 'text': 'Em 25 de novembro de 2025, a cotação de fechamento PTAX do Euro foi de 6.2244 para compra e 6.2256 para venda.', 'extras': {'signature': 'CpAFAXLI2nxTYkkjSYmWEYduBf9MuE9LprjSEJg2HgRzLVstdJmzPl2ueTS74XAX5p6bAXVLMA+QjETV00mOWG0y1Pw7KSS+ap3/vrzv1EOAnWZZuxPz7CQ7r7u0ntJ8a4ukizSPAzPocmeabBooSfjgfgG74zFKWVuU5xUTG+UJEniIjnDPVe226yp1m7p2iVI1mzTiiEYWe4HV6zKhrv+SNENAWZkzGmHiZ7Eu4WSi0qzMeGYEBly0HmeR35oDav5PS0Bnnoudpbsd7DcSjJaQtM+IExtaJT+HazId13KlJRfsA9HpSc3ZlM4ULNLsFKpMM9F7aiefsE04QpeHYewSCKQ9buEprnz9hBJYFlNRQEc0qMfNO10SLG4lyQlcW2r1QBlowFgVE6+Q2jhz5MyAJBNe8+oQFAwUnkqjOM2GUGchPi5Zi5qlwH048UU6KvD8FeBhcKdrKrR9KHSq0URl4Sng4MqLpmbV7XCYkekQw84jrl9sZYUP+MAIJ42H8AtvVmNOGgZH2sSshcuZh9WgdTFEKjnRqMdAENKUktR5alJkjgKUS2QDR+tAAV8dL+Lby+6ELu0V2LcvFcexw+doBpid6LNIQ6HfzwZFF/IAYkESn4iwiH1T77KrwMHH6rS1Tr5cv64eSsE7AjxyoMZxugemPHwZzpmjioGRMwWUGbGc24wZce3OXFWUY1IWRDmN7B2OUzez4LTG4h2mpnnVtw41uXEACdX+W5En0sTimEdWWsRViLLOc9AVRDnAEqNLctdxU+BZBVzMOBpYCI6c